In [1]:
!pip install trustgraph-base

  Using cached trustgraph_base-0.16.4-py3-none-any.whl
  Using cached pulsar_client-3.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.0 kB)
Using cached pulsar_client-3.5.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import trustgraph.api as tg
from trustgraph.knowledge import DigitalDocument, Organization
from trustgraph.knowledge import PublicationEvent, hash, to_uri
from trustgraph.knowledge import PREF_PUBEV, PREF_ORG, PREF_DOC

In [3]:
# Open a text document from this repo.  PDFs are binary blobs
pdf = open("../sources/Challenger-Report-Vol1.pdf", "rb").read()

In [4]:
cli = tg.Api()

In [5]:
org_id = to_uri(PREF_ORG, "1dd51ece-8bd3-48b8-98ce-1ac9164c5214")
doc_id = to_uri(PREF_DOC, "72ef3374-af7a-40c4-8c7b-45050aef5b90")
pub_id = to_uri(PREF_PUBEV, "59012ae1-65d4-441f-8288-b6f3c6c15333")

In [6]:
org = Organization(
    id = org_id,
    name = "NASA",
)

pubev = PublicationEvent(
    id = pub_id,
    description = "Presidential commission publication",
    start_date = "1986-06-06",
    end_date = "1986-06-06",
    organization = org,
)

doc = DigitalDocument(
    id = doc_id,
    name = "Challenger Report Volume 1",
    description = "The findings of the Presidential Commission regarding the circumstances surrounding the Challenger accident are reported and recommendations for corrective action are outlined",
    copyright_holder = "US Government",
    copyright_notice = "Work of the US Gov. Public Use Permitted",
    copyright_year = "1986",
    keywords = ["nasa", "challenger", "space-shuttle", "shuttle", "orbiter"],
    publication = pubev,
    url = "https://ntrs.nasa.gov/citations/19860015255",
)

In [7]:
cli.load_document(pdf, doc_id, doc)